In [1]:
#importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
#open the browser
driver=webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
time.sleep(3)
driver.maximize_window()
wait_time = WebDriverWait(driver, 20)

def get_kerala_bus_data(xpath):
    bus_links = []  # Holds all the bus URLs
    bus_routes = []  # Holds all the route names
    
    for page_num in range(1, 3):
        route_elements = driver.find_elements(By.XPATH, xpath)
        bus_links.extend([element.get_attribute("href") for element in route_elements])
        bus_routes.extend([element.text for element in route_elements])
        
        try:
            pagination_table = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]'))
            )
            next_page_button = pagination_table.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()="{page_num + 1}"]')
            
            # Scroll the next page button into view
            ActionChains(driver).move_to_element(next_page_button).perform()
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable(next_page_button))
            next_page_button.click()
            
        except NoSuchElementException:
            print(f"End of pages detected at page {page_num}")
            break
        except ElementClickInterceptedException:
            print(f"Could not click the next page button at page {page_num}, trying again.")
            time.sleep(1)  # Wait a moment before retrying
            next_page_button.click()
    
    return bus_links, bus_routes

In [5]:
# change dataframe to csv
path=r"C:\Users\Vanathi\Documents\redbus scraped details\kerala_df.csv"
kerala_df.to_csv(path,index=False)

In [4]:
# Run the function to get the bus links and routes
bus_links, bus_routes = get_kerala_bus_data("//a[@class='route']")

# Now create the DataFrame with the returned variables
kerala_df = pd.DataFrame({
    "Bus_Route_Name": bus_routes,  # variable name from ROUTE_KERALA
    "Bus_Route_Link": bus_links   # variable name from LINKS_KERALA
})

# Display the DataFrame
print(kerala_df)


End of pages detected at page 2
                     Bus_Route_Name  \
0            Kozhikode to Ernakulam   
1            Bangalore to Kozhikode   
2            Kozhikode to Bangalore   
3            Ernakulam to Kozhikode   
4               Kozhikode to Mysore   
5               Mysore to Kozhikode   
6   Kozhikode to Thiruvananthapuram   
7             Kozhikode to Thrissur   
8   Thiruvananthapuram to Kozhikode   
9      Bangalore to Kannur (Kerala)   
10            Thrissur to Kozhikode   
11     Kannur (Kerala) to Bangalore   
12   Kalpetta (kerala) to Bangalore   
13   Bangalore to Kalpetta (kerala)   
14            Kozhikode to Kottayam   
15               Kozhikode to Aluva   
16   Kalpetta (kerala) to Kozhikode   

                                       Bus_Route_Link  
0   https://www.redbus.in/bus-tickets/kozhikode-to...  
1   https://www.redbus.in/bus-tickets/bangalore-to...  
2   https://www.redbus.in/bus-tickets/kozhikode-to...  
3   https://www.redbus.in/bus-tickets/ern

In [6]:
df_1=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\kerala_df.csv")

In [7]:
print(df_1.columns)

Index(['Bus_Route_Name', 'Bus_Route_Link'], dtype='object')


In [8]:
driver_kerala = webdriver.Chrome()
Busname_1 = []
Bustype_1 = []
Starttime_1 = []
Endtime_1 = []
Ratings_1 = []
Total_duration_1 = []
Prices_1 = []
Seats_available_1 = []
Route_names = []
Route_links = []

for i, r in df_1.iterrows():
    link = r["Bus_Route_Link"]
    routes = r["Bus_Route_Name"]

    # Open the link
    driver_kerala.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    elements = driver_kerala.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # Click elements to view bus details
    try:
        clicks = driver_kerala.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
    time.sleep(2)

    # Scroll through the page to load all bus details
    scrolling = True
    while scrolling:
        # Get the current scroll height
        last_height = driver_kerala.execute_script("return document.body.scrollHeight")

        # Scroll down to the bottom
        driver_kerala.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new content to load
        time.sleep(5)

        # Get the new scroll height
        new_height = driver_kerala.execute_script("return document.body.scrollHeight")

        # Check if the page height has changed
        if new_height == last_height:
            scrolling = False

    # Extract bus details
    try:
        bus_name = WebDriverWait(driver_kerala, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='travels lh-24 f-bold d-color']"))
        )
        bus_type = driver_kerala.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver_kerala.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver_kerala.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver_kerala.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        try:
            rating = driver_kerala.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        except:
            continue
        price = driver_kerala.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver_kerala.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for bus in bus_name:
            Busname_1.append(bus.text)
            Route_links.append(link)
            Route_names.append(routes)
        for bus_type_elem in bus_type:
            Bustype_1.append(bus_type_elem.text)
        for start_time_elem in start_time:
            Starttime_1.append(start_time_elem.text)
        for end_time_elem in end_time:
            Endtime_1.append(end_time_elem.text)
        for total_duration_elem in total_duration:
            Total_duration_1.append(total_duration_elem.text)
        for ratings in rating:
            Ratings_1.append(ratings.text)
        for price_elem in price:
            Prices_1.append(price_elem.text)
        for seats_elem in seats:
            Seats_available_1.append(seats_elem.text)
    except Exception as e:
        print(f"Error occurred while extracting details: {e}")
        continue

print("Successfully Completed")



Successfully Completed


In [9]:
# Before converting to DataFrame, check if all lists have the same length
lengths = {
    'Busname_1': len(Busname_1),
    'Bustype_1': len(Bustype_1),
    'Starttime_1': len(Starttime_1),
    'Endtime_1': len(Endtime_1),
    'Total_duration_1': len(Total_duration_1),
    'Prices_1': len(Prices_1),
    'Seats_available_1': len(Seats_available_1),
    'Route_names': len(Route_names),
    'Route_links': len(Route_links),
    'Ratings_1': len(Ratings_1)
}

print(lengths)

# Ensure that all lists have the same length
max_length = max(lengths.values())  # Find the longest list
for lst in [Busname_1, Bustype_1, Starttime_1, Endtime_1, Total_duration_1, Prices_1, Seats_available_1, Route_names, Route_links, Ratings_1]:
    while len(lst) < max_length:
        lst.append(None)  # Or you can append a placeholder like an empty string




{'Busname_1': 884, 'Bustype_1': 884, 'Starttime_1': 884, 'Endtime_1': 884, 'Total_duration_1': 884, 'Prices_1': 884, 'Seats_available_1': 884, 'Route_names': 884, 'Route_links': 884, 'Ratings_1': 323}


In [10]:
# Convert list to DataFrame
data = {
    'Bus_name': Busname_1,
    'Bus_type': Bustype_1,
    'Start_time': Starttime_1,
    'End_time': Endtime_1,
    'Total_duration': Total_duration_1,
    'Price': Prices_1,
    "Seats_Available": Seats_available_1,
    "Ratings": Ratings_1,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_1 = pd.DataFrame(data)

# Convert DataFrame to CSV
path = r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_1.csv"
df_buses_1.to_csv(path, index=False)

print("Successfully Completed and saved to CSV.")

Successfully Completed and saved to CSV.


In [11]:
a=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_1.csv")
a

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,KSRTC (Kerala) - 2820,Super Fast Non AC Seater (2+3),00:55,05:50,04h 55m,INR 269,5 Seats available,3.8,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
1,KSRTC (Kerala) - 2495,Low Floor AC Seater 2+2,06:00,11:20,05h 20m,INR 383,2 Seats available,4.4\n106,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
2,KSRTC (Kerala) - 794,Super Fast Non AC Seater (2+3),06:40,12:30,05h 50m,INR 269,43 Seats available,3.9\n132,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
3,KSRTC (Kerala) - 2710,Super Fast Non AC Seater (2+3),07:20,13:06,05h 46m,INR 269,44 Seats available,3.0\n35,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
4,KSRTC (Kerala) - 3597,Low Floor AC Seater 2+2,08:00,13:20,05h 20m,INR 383,13 Seats available,4.3\n92,https://www.redbus.in/bus-tickets/kozhikode-to...,Kozhikode to Ernakulam
...,...,...,...,...,...,...,...,...,...,...
879,Sonia Travels,A/C Seater/Sleeper (2+1),21:30,23:29,01h 59m,INR 450,42 Seats available,NaN,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode
880,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),06:32,09:00,02h 28m,INR 350,13 Seats available,NaN,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode
881,A1 Travels,Bharat Benz A/C Sleeper (2+1),21:00,23:30,02h 30m,INR 1100,34 Seats available,NaN,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode
882,KP Travels,Scania Multi-Axle AC Semi Sleeper (2+2),15:30,18:00,02h 30m,INR 2000,53 Seats available,NaN,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode
